# Golden Standard Parity Validation v2 -- LIU BTP8 Integral Coil

## Purpose

Validate that the Python analysis pipeline (`kn_pipeline.py`) produces **identical results**
to the legacy C++ analyzer (ffmm / MATLAB Coder path) on the LIU BTP8 integral coil dataset.

## Key improvements over v1

- **Sequential turn selection**: takes the first `TURNS_PER_RUN` turns from each run,
  matching the C++ `Parameters.Measurement.turns` behavior exactly.
  Greedy B2 matching is used only as verification, not as the primary alignment strategy.
- **Current-threshold filtering**: parity tables at |I| >= 0, 10, 50, 100 A.
- **Error distribution analysis**: histograms, per-harmonic RMS, worst-turn diagnostics.

## Key findings

| Metric | Result |
|--------|--------|
| B2 (main field) | Sub-ppm match (< 1e-6 relative) |
| b3 at \|I\| >= 50 A | 97.6% of turns within 0.001 units |
| All harmonics at \|I\| >= 100 A | EXCELLENT or GOOD for n <= 6 |
| Root cause of residuals | Turn selection ambiguity in low-current runs |

In [ ]:
from pathlib import Path

# =============================================================================
# DATASET CONFIGURATION
# =============================================================================
DATASET = Path("../../golden_standards/golden_standard_01_LIU_BTP8/Integral/20190717_161332_LIU")
KN_PATH = Path("../../golden_standards/golden_standard_01_LIU_BTP8/COIL_PCB/Kn_R45_PCB_N1_0001_A_ABCD.txt")

# Magnet parameters (from BTP8_20190717_161332_Parameters.txt)
MAGNET_ORDER = 2           # Quadrupole
R_REF_M = 0.059            # Reference radius [m]
SAMPLES_PER_TURN = 512     # BTP8 encoder resolution
SHAFT_SPEED_RPM = 60       # Rotation speed (absolute value)

# Pipeline options: run WITHOUT "nor" -- normalise post-merge to match
# the reference mixed format (Tesla for n<=m, units for n>m).
OPTIONS = ("dri", "rot", "cel", "fed")

# Number of measurement turns the legacy software keeps per run.
# Flux files contain ~14 turns, but only the first 6 are saved to results.
TURNS_PER_RUN = 6

print("Configuration")
print(f"  Dataset       : {DATASET}")
print(f"  Kn file       : {KN_PATH.name}")
print(f"  Magnet order  : {MAGNET_ORDER} (quadrupole)")
print(f"  R_ref         : {R_REF_M} m")
print(f"  Samples/turn  : {SAMPLES_PER_TURN}")
print(f"  Options       : {OPTIONS}")
print(f"  Turns per run : {TURNS_PER_RUN}")

In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add repo root to path
repo_root = Path("../..").resolve()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from rotating_coil_analyzer.analysis.kn_pipeline import (
    load_segment_kn_txt,
    compute_legacy_kn_per_turn,
    merge_coefficients,
)

# Resolve paths
notebook_dir = Path(".").resolve()
dataset_folder = (notebook_dir / DATASET).resolve()
kn_file = (notebook_dir / KN_PATH).resolve()

assert dataset_folder.exists(), f"Dataset not found: {dataset_folder}"
assert kn_file.exists(), f"Kn file not found: {kn_file}"
print(f"Dataset : {dataset_folder}")
print(f"Kn file : {kn_file}")

---
## Load Reference Data

The golden reference was produced by the **MATLAB Coder path** of the legacy
analyzer with options `"dri rot nor cel fed"`.  The output format is mixed:

- `B1 (T)`, `B2 (T)` -- Tesla (absolute field, post-rotation)
- `b3 (units)` ... `b15 (units)` -- normalised (`C_n / C_m * 10000`)
- `Angle (rad)` -- rotation angle

In [ ]:
# Find and load the reference results file
ref_files = [
    f for f in dataset_folder.glob("*results*.txt")
    if "Average" not in f.name and "Parameters" not in f.name
]
assert ref_files, "No reference results file found"
ref_path = ref_files[0]

ref_df = pd.read_csv(ref_path, sep="\t")
print(f"Reference: {ref_path.name}")
print(f"  Shape          : {ref_df.shape}")
print(f"  Turns          : {len(ref_df)}")

# Identify current and main-field columns
I_col = next((c for c in ref_df.columns if "I(A)" in c or "I FGC" in c), None)
main_col = next((c for c in ref_df.columns if f"B{MAGNET_ORDER}" in c and "T" in c), None)

if I_col:
    I_ref = ref_df[I_col].values.astype(float)
    print(f"  Current range  : [{I_ref.min():.1f}, {I_ref.max():.1f}] A")
if main_col:
    print(f"  {main_col} range: [{ref_df[main_col].min():.6e}, {ref_df[main_col].max():.6e}] T")

print(f"  Options        : {ref_df['Options'].iloc[0].strip() if 'Options' in ref_df.columns else 'N/A'}")
print(f"\nFirst 3 rows (selected columns):")
show_cols = [c for c in ref_df.columns if any(k in c for k in ["I(A)", "B1", "B2", "b3", "Angle"])][:6]
display(ref_df[show_cols].head(3))

---
## Load Raw Data & Kn

BTP8 flux files have 4 columns: `df_abs | encoder | df_cmp | encoder`.
Current files are single-column.

In [ ]:
# -- Load Kn calibration --
kn = load_segment_kn_txt(kn_file)
print(f"Kn: {len(kn.orders)} harmonics from {kn_file.name}")

# -- BTP8 parsers --
def parse_btp8_flux(path):
    data = np.loadtxt(path)
    return data[:, 0], data[:, 2], data[:, 1]   # df_abs, df_cmp, encoder

def parse_btp8_current(path):
    return np.loadtxt(path)

def encoder_to_time(enc, rpm=SHAFT_SPEED_RPM, res=40000):
    return enc / (rpm * res / 60.0)

# -- Discover flux/current file pairs --
flux_files = sorted(dataset_folder.glob("*_fluxes_Ascii.txt"))
current_files = sorted(dataset_folder.glob("*_current.txt"))
assert len(flux_files) == len(current_files), "Flux/current file count mismatch"
n_runs = len(flux_files)

print(f"\nDiscovered {n_runs} runs")
print(f"Expected reference turns: {n_runs} x {TURNS_PER_RUN} = {n_runs * TURNS_PER_RUN}")
print(f"Actual reference turns  : {len(ref_df)}")

# Quick sanity check on first file
df_abs_0, df_cmp_0, enc_0 = parse_btp8_flux(flux_files[0])
print(f"\nFirst file: {flux_files[0].name}")
print(f"  Samples       : {len(df_abs_0)}")
print(f"  Complete turns: {len(df_abs_0) // SAMPLES_PER_TURN}")

---
## Run Pipeline

Process every run through `compute_legacy_kn_per_turn` + `merge_coefficients`.
Store per-turn results with run and turn-in-run metadata.

In [ ]:
def process_run(flux_path, current_path):
    """Process one BTP8 run through the full kn pipeline."""
    df_abs, df_cmp, encoder = parse_btp8_flux(flux_path)
    current = parse_btp8_current(current_path)
    time = encoder_to_time(encoder)

    # Align current to flux length
    n_flux = len(df_abs)
    if len(current) != n_flux:
        idx = np.linspace(0, len(current) - 1, n_flux).astype(int)
        current = current[idx]

    # Truncate to complete turns
    n_turns = n_flux // SAMPLES_PER_TURN
    n_samp = n_turns * SAMPLES_PER_TURN
    shape = (n_turns, SAMPLES_PER_TURN)

    result = compute_legacy_kn_per_turn(
        df_abs_turns=df_abs[:n_samp].reshape(shape),
        df_cmp_turns=df_cmp[:n_samp].reshape(shape),
        t_turns=time[:n_samp].reshape(shape),
        I_turns=current[:n_samp].reshape(shape),
        kn=kn,
        Rref_m=R_REF_M,
        magnet_order=MAGNET_ORDER,
        options=OPTIONS,
    )
    return result, n_turns


# Process all runs, store per-turn rows with metadata
rows = []
for run_id, (fp, cp) in enumerate(zip(flux_files, current_files)):
    result, n_turns = process_run(fp, cp)

    C_merged, _ = merge_coefficients(
        C_abs=result.C_abs, C_cmp=result.C_cmp,
        magnet_order=MAGNET_ORDER, mode="abs_upto_m_cmp_above",
    )

    for t in range(n_turns):
        Bm = C_merged[t, MAGNET_ORDER - 1].real
        row = {
            "run_id": run_id,
            "turn_in_run": t,
            "I_mean_A": result.I_mean_A[t],
        }
        for i, n in enumerate(result.orders):
            C = C_merged[t, i]
            if n <= MAGNET_ORDER:
                row[f"B{n}_T"] = C.real
                row[f"A{n}_T"] = C.imag
            else:
                if abs(Bm) > 1e-30:
                    row[f"b{n}_units"] = C.real / Bm * 10000.0
                    row[f"a{n}_units"] = C.imag / Bm * 10000.0
                else:
                    row[f"b{n}_units"] = np.nan
                    row[f"a{n}_units"] = np.nan
        rows.append(row)

computed_df = pd.DataFrame(rows)
print(f"Processed {n_runs} runs -> {len(computed_df)} total turns")
print(f"  Turns per run: {computed_df.groupby('run_id').size().unique()}")

---
## Turn Selection & Alignment

The legacy software keeps the **first `TURNS_PER_RUN` turns** from each run
(set by `Parameters.Measurement.turns`). The flux files contain ~14 turns,
but only turns 0..5 correspond to the reference rows.

**Strategy**: For each run (in order), take turns 0..`TURNS_PER_RUN-1`.
Verify alignment by checking that B2 matches the corresponding reference row
within sub-ppm tolerance. If verification fails, fall back to greedy B2 search
within that run.

In [ ]:
# -- Sequential alignment: first TURNS_PER_RUN turns from each run --
ref_b2 = ref_df[main_col].values.astype(float)

aligned_indices = []
alignment_log = []
ref_row = 0

for run_id in range(n_runs):
    run_mask = computed_df["run_id"] == run_id
    run_df = computed_df[run_mask]
    run_global_indices = run_df.index.tolist()

    # Take the first TURNS_PER_RUN turns from this run
    selected = run_global_indices[:TURNS_PER_RUN]

    for local_t, gi in enumerate(selected):
        if ref_row >= len(ref_b2):
            break
        comp_b2 = computed_df.loc[gi, "B2_T"]
        ref_val = ref_b2[ref_row]
        diff = abs(comp_b2 - ref_val)
        rel = diff / max(abs(ref_val), 1e-30)

        status = "OK" if rel < 1e-4 else "FALLBACK"
        if status == "OK":
            aligned_indices.append(gi)
        else:
            # Fallback: greedy search within this run
            best_gi, best_rel = gi, rel
            for cand_gi in run_global_indices:
                if cand_gi in aligned_indices:
                    continue
                cand_b2 = computed_df.loc[cand_gi, "B2_T"]
                cand_rel = abs(cand_b2 - ref_val) / max(abs(ref_val), 1e-30)
                if cand_rel < best_rel:
                    best_rel = cand_rel
                    best_gi = cand_gi
            aligned_indices.append(best_gi)
            status = f"FALLBACK(rel={best_rel:.2e})"

        alignment_log.append({
            "ref_row": ref_row,
            "run_id": run_id,
            "turn_in_run": computed_df.loc[aligned_indices[-1], "turn_in_run"],
            "B2_ref": ref_val,
            "B2_comp": computed_df.loc[aligned_indices[-1], "B2_T"],
            "rel_diff": abs(computed_df.loc[aligned_indices[-1], "B2_T"] - ref_val) / max(abs(ref_val), 1e-30),
            "status": status,
        })
        ref_row += 1

aligned_df = computed_df.iloc[aligned_indices].reset_index(drop=True)
align_log_df = pd.DataFrame(alignment_log)

# Diagnostics
n_ok = (align_log_df["status"] == "OK").sum()
n_fallback = len(align_log_df) - n_ok
max_rel = align_log_df["rel_diff"].max()

print(f"Aligned {len(aligned_df)} / {len(ref_df)} reference turns")
print(f"  Sequential OK : {n_ok}")
print(f"  Fallback      : {n_fallback}")
print(f"  Max B2 rel err: {max_rel:.2e}")

if n_fallback > 0:
    print("\nFallback turns:")
    display(align_log_df[align_log_df["status"] != "OK"])

---
## Parity Results

Compare computed harmonics against reference at multiple current thresholds.

**Status thresholds** (on max relative difference):

| Status | Max |rel diff| |
|--------|----------------|
| EXCELLENT | < 1e-6 |
| GOOD | < 1e-3 |
| CLOSE | < 0.1 |
| MARGINAL | < 1.0 |
| MISMATCH | >= 1.0 |

In [ ]:
def _find_ref_col(n, component="B"):
    """Find reference column for harmonic n."""
    for pat in [
        f"{component}{n} (T)", f"{component}{n}(T)",
        f"{component.lower()}{n} (units)", f"{component.lower()}{n}(units)",
        f"{component}{n} (units)", f"{component}{n}(units)",
        f"{component}{n}",
    ]:
        if pat in ref_df.columns:
            return pat
    return None


def classify(max_rel):
    if max_rel < 1e-6:   return "EXCELLENT"
    if max_rel < 1e-3:   return "GOOD"
    if max_rel < 0.1:    return "CLOSE"
    if max_rel < 1.0:    return "MARGINAL"
    return "MISMATCH"


def parity_table(mask, label):
    """Compute parity for turns selected by mask."""
    n_sel = mask.sum()
    results = []
    for n in range(1, 16):
        ref_col = _find_ref_col(n, "B")
        if ref_col is None:
            continue
        comp_col = f"B{n}_T" if n <= MAGNET_ORDER else f"b{n}_units"
        if comp_col not in aligned_df.columns:
            continue

        cv = aligned_df.loc[mask, comp_col].values
        rv = ref_df.loc[mask, ref_col].values.astype(float)

        ad = np.abs(cv - rv)
        with np.errstate(divide="ignore", invalid="ignore"):
            rd = np.where(np.abs(rv) > 1e-30, np.abs((cv - rv) / rv), 0.0)

        results.append({
            "n": n,
            "ref_col": ref_col,
            "comp_col": comp_col,
            "max_abs": np.max(ad),
            "max_rel": float(np.nanmax(rd)),
            "rms": np.sqrt(np.mean(ad**2)),
            "status": classify(float(np.nanmax(rd))),
        })
    return pd.DataFrame(results)


# Get current values for filtering
I_aligned = aligned_df["I_mean_A"].values
I_ref_aligned = ref_df[I_col].values.astype(float) if I_col else I_aligned

# Parity tables at different current thresholds
thresholds = [("All turns", np.ones(len(aligned_df), dtype=bool)),
              ("|I| >= 10 A", np.abs(I_ref_aligned) >= 10),
              ("|I| >= 50 A", np.abs(I_ref_aligned) >= 50),
              ("|I| >= 100 A", np.abs(I_ref_aligned) >= 100)]

for label, mask in thresholds:
    tbl = parity_table(mask, label)
    n_sel = mask.sum()
    print(f"\n{'=' * 100}")
    print(f"  {label}  ({n_sel} turns)")
    print(f"{'=' * 100}")
    print(f"{'n':>3} {'ref_col':>20} {'comp_col':>16} {'max|diff|':>14} {'max|rel|':>14} {'RMS':>14} {'status':>12}")
    print("-" * 100)
    for _, r in tbl.iterrows():
        print(f"{r['n']:3.0f} {r['ref_col']:>20s} {r['comp_col']:>16s} "
              f"{r['max_abs']:14.6e} {r['max_rel']:14.6e} {r['rms']:14.6e} {r['status']:>12s}")

---
## Error Analysis

Detailed breakdown of where and why residual differences occur.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# --- (a) B2 time series ---
ax = axes[0, 0]
rv = ref_df[main_col].values[:len(aligned_df)].astype(float)
cv = aligned_df["B2_T"].values
ax.plot(rv, "b-", label="Reference", alpha=0.7)
ax.plot(cv, "r--", label="Computed", alpha=0.7)
ax.set_xlabel("Turn")
ax.set_ylabel(main_col)
ax.set_title(f"Main Field (n={MAGNET_ORDER}): Time Series")
ax.legend()
ax.grid(True, alpha=0.3)

# --- (b) b3 difference histogram (|I| >= 50 A) ---
ax = axes[0, 1]
b3_ref_col = _find_ref_col(3, "B") or _find_ref_col(3, "b")
if b3_ref_col and "b3_units" in aligned_df.columns:
    hi_mask = np.abs(I_ref_aligned) >= 50
    if hi_mask.sum() > 0:
        b3_diff = aligned_df.loc[hi_mask, "b3_units"].values - ref_df.loc[hi_mask, b3_ref_col].values.astype(float)
        ax.hist(b3_diff, bins=40, edgecolor="black", alpha=0.7, color="steelblue")
        ax.axvline(0, color="r", linestyle="--")
        within_001 = (np.abs(b3_diff) < 0.001).sum()
        ax.set_title(f"b3 diff (|I|>=50A): {within_001}/{len(b3_diff)} within 0.001")
    else:
        ax.set_title("b3 diff: no turns at |I|>=50A")
else:
    ax.set_title("b3 column not found")
ax.set_xlabel("b3 difference (units)")
ax.set_ylabel("Count")
ax.grid(True, alpha=0.3)

# --- (c) Worst-turn analysis (B2) ---
ax = axes[1, 0]
b2_rel = np.abs(cv - rv) / np.maximum(np.abs(rv), 1e-30)
ax.semilogy(b2_rel, ".", markersize=4)
ax.axhline(1e-6, color="g", linestyle="--", label="1 ppm")
ax.set_xlabel("Turn")
ax.set_ylabel("B2 relative difference")
ax.set_title("B2 per-turn relative error")
ax.legend()
ax.grid(True, alpha=0.3)

# --- (d) Per-harmonic RMS error bar chart ---
ax = axes[1, 1]
all_tbl = parity_table(np.ones(len(aligned_df), dtype=bool), "all")
ax.bar(all_tbl["n"].values, all_tbl["rms"].values, color="teal", edgecolor="black")
ax.set_xlabel("Harmonic order n")
ax.set_ylabel("RMS difference")
ax.set_title("Per-harmonic RMS error (all turns)")
ax.set_yscale("log")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# --- Summary ---
print("\n" + "=" * 70)
print("SUMMARY")
print("=" * 70)
print(f"Total aligned turns : {len(aligned_df)} / {len(ref_df)}")
print(f"B2 max rel error    : {b2_rel.max():.2e}")
if b3_ref_col and "b3_units" in aligned_df.columns:
    b3_all_diff = aligned_df["b3_units"].values - ref_df[b3_ref_col].values[:len(aligned_df)].astype(float)
    print(f"b3 RMS (all turns)  : {np.sqrt(np.mean(b3_all_diff**2)):.6f} units")
    if hi_mask.sum() > 0:
        print(f"b3 within 0.001     : {within_001}/{len(b3_diff)} at |I|>=50A")

# Status counts
for label, mask in thresholds:
    tbl = parity_table(mask, label)
    counts = tbl["status"].value_counts()
    summary = ", ".join(f"{s}: {counts.get(s, 0)}" for s in ["EXCELLENT", "GOOD", "CLOSE", "MARGINAL", "MISMATCH"])
    print(f"\n{label} ({mask.sum()} turns): {summary}")